In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pymongo
from splinter import Browser
import pandas as pd
import numpy as np
import time
import os
import requests

In [2]:
url = 'https://data.bitcoinity.org/markets/exchanges/all/2y#rank_desc'
browser = Browser('chrome')
browser.visit(url)
browser.find_by_css('a.v_all').first.click()
time.sleep(15)
page = browser.html
soup = BeautifulSoup(page, 'lxml')

In [3]:
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df=df.rename(columns={ df.columns[2]: "Currency",df.columns[4]: "Trade_Volume",df.columns[9]: "Trade_per_min"})
df=df.drop(df.columns[[0,3,5,6,7,8]], axis=1)
df.head()

,Name,Currency,Trade_Volume,Trade_per_min
0,E Bitfinex,USD,"4 19,943,916",4 27.84
1,E Bitstamp,USD,6822443,10.66
2,D BitMEX,USD,2051910,1.12
3,E bitFlyer,JPY,"5 12,609,309",24.63
4,E GDAX,USD,8080511,5 26.63


In [4]:
df_sum=df.groupby(['Currency'])['Trade_Volume'].agg([np.sum, len])
df_sum=df_sum.rename(columns={ df_sum.columns[0]: "Trade_Volume",df_sum.columns[1]: "Exchange_count"})
df_sum.head()

,Trade_Volume,Exchange_count
Currency,,
AED,7088,1
ARS,5698,1
AUD,22596183431,2
BRL,18464810423,2
CAD,3092002000566944242329053,6


In [5]:
codeToCountry = {
  "AED" : "United Arab Emirates",
  "AFN" : "Afghanistan",
  "ALL" : "Albania",
  "AMD" : "Armenia",
  "ANG" : "Netherlands Antilles",
  "AOA" : "Angola",
  "ARS" : "Argentina",
  "AUD" : "Australia",
  "AWG" : "Aruba",
  "AZN" : "Azerbaijan",
  "BAM" : "Bosnia and Herzegovina",
  "BBD" : "Barbados",
  "BDT" : "Bangladesh",
  "BGN" : "Bulgaria",
  "BHD" : "Bahrain",
  "BIF" : "Burundi",
  "BMD" : "Bermuda",
  "BND" : "Brunei",
  "BOB" : "Bolivia",
  "BOV" : "Bolivia",
  "BRL" : "Brazil",
  "BSD" : "Bahamas",
  "BTN" : "Bhutan",
  "BWP" : "Botswana",
  "BYR" : "Belarus",
  "BZD" : "Belize",
  "CAD" : "Canada",
  "CDF" : "Democratic Republic of Congo",
  "CHE" : "Switzerland",
  "CHF" : "Switzerland",
  "CHW" : "Switzerland",
  "CLF" : "Chile",
  "CLP" : "Chile",
  "CNY" : "China",
  "COP" : "Colombia",
  "COU" : "Colombia",
  "CRC" : "Costa Rica",
  "CUP" : "Cuba",
  "CVE" : "Cape Verde",
  "CYP" : "Cyprus",
  "CZK" : "Czech Republic",
  "DJF" : "Djibouti",
  "DKK" : "Denmark",
  "DOP" : "Dominican Republic",
  "DZD" : "Algeria",
  "EEK" : "Estonia",
  "EGP" : "Egypt",
  "ERN" : "Eritrea",
  "ETB" : "Ethiopia",
  "EUR" : "European Union",
  "FJD" : "Fiji",
  "FKP" : "Falkland Islands",
  "GBP" : "United Kingdom",
  "GEL" : "Georgia",
  "GHS" : "Ghana",
  "GIP" : "Gibraltar",
  "GMD" : "Gambia",
  "GNF" : "Guinea",
  "GTQ" : "Guatemala",
  "GYD" : "Guyana",
  "HKD" : "Hong Kong",
  "HNL" : "Honduras",
  "HRK" : "Croatia",
  "HTG" : "Haiti",
  "HUF" : "Hungary",
  "IDR" : "Indonesia",
  "ILS" : "Israel",
  "INR" : "India",
  "IQD" : "Iraq",
  "IRR" : "Iran",
  "ISK" : "Iceland",
  "JMD" : "Jamaica",
  "JOD" : "Jordan",
  "JPY" : "Japan",
  "KES" : "Kenya",
  "KGS" : "Kyrgyzstan",
  "KHR" : "Cambodia",
  "KMF" : "Comoros",
  "KPW" : "North Korea",
  "KRW" : "South Korea",
  "KWD" : "Kuwait",
  "KYD" : "Cayman Islands",
  "KZT" : "Kazakhstan",
  "LAK" : "Laos",
  "LBP" : "Lebanon",
  "LKR" : "Sri Lanka",
  "LRD" : "Liberia",
  "LSL" : "Lesotho",
  "LTL" : "Lithuania",
  "LVL" : "Latvia",
  "LYD" : "Libya",
  "MAD" : "Morocco, Western Sahara",
  "MDL" : "Moldova",
  "MGA" : "Madagascar",
  "MKD" : "Former Yugoslav Republic of Macedonia",
  "MMK" : "Myanmar",
  "MNT" : "Mongolia",
  "MOP" : "Macau Special Administrative Region",
  "MRO" : "Mauritania",
  "MTL" : "Malta",
  "MUR" : "Mauritius",
  "MVR" : "Maldives",
  "MWK" : "Malawi",
  "MXN" : "Mexico",
  "MXV" : "Mexico",
  "MYR" : "Malaysia",
  "MZN" : "Mozambique",
  "NAD" : "Namibia",
  "NGN" : "Nigeria",
  "NIO" : "Nicaragua",
  "NOK" : "Norway",
  "NPR" : "Nepal",
  "NZD" : "New Zealand",
  "OMR" : "Oman",
  "PAB" : "Panama",
  "PEN" : "Peru",
  "PGK" : "Papua New Guinea",
  "PHP" : "Philippines",
  "PKR" : "Pakistan",
  "PLN" : "Poland",
  "PYG" : "Paraguay",
  "QAR" : "Qatar",
  "RON" : "Romania",
  "RSD" : "Serbia",
  "RUB" : "Russia",
  "RUR" : "Russia",
  "RWF" : "Rwanda",
  "SAR" : "Saudi Arabia",
  "SBD" : "Solomon Islands",
  "SCR" : "Seychelles",
  "SDG" : "Sudan",
  "SEK" : "Sweden",
  "SGD" : "Singapore",
  "SHP" : "Saint Helena",
  "SKK" : "Slovakia",
  "SLL" : "Sierra Leone",
  "SOS" : "Somalia",
  "SRD" : "Suriname",
  "STD" : "São Tomé and Príncipe",
  "SYP" : "Syria",
  "SZL" : "Swaziland",
  "THB" : "Thailand",
  "TJS" : "Tajikistan",
  "TMM" : "Turkmenistan",
  "TND" : "Tunisia",
  "TOP" : "Tonga",
  "TRY" : "Turkey",
  "TTD" : "Trinidad and Tobago",
  "TWD" : "Taiwan",
  "TZS" : "Tanzania",
  "UAH" : "Ukraine",
  "UGX" : "Uganda",
  "USD" : "United States",
  "UYU" : "Uruguay",
  "UZS" : "Uzbekistan",
  "VEB" : "Venezuela",
  "VEF" : "Venezuela",
  "VND" : "Vietnam",
  "VUV" : "Vanuatu",
  "WST" : "Samoa",
  "XAF" : "Cameroon, Central African Republic, Congo, Chad, Equatorial Guinea, Gabon",
  "XCD" : "Anguilla, Antigua and Barbuda, Dominica, Grenada, Montserrat, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines",
  "XDR" : "International Monetary Fund",
  "XFO" : "Bank for International Settlements",
  "XFU" : "International Union of Railways",
  "XOF" : "Benin, Burkina Faso, Côte d'Ivoire, Guinea-Bissau, Mali, Niger, Senegal, Togo",
  "XPF" : "French Polynesia, New Caledonia, Wallis and Futuna",
  "YER" : "Yemen",
  "ZAR" : "South Africa",
  "ZMK" : "Zambia",
  "ZWD" : "Zimbabwe"}

In [6]:
country=pd.DataFrame(list(codeToCountry.items()), columns=['Currency','Country'])
country=country.set_index(['Currency'])

In [7]:
country.head()

,Country
Currency,
AED,United Arab Emirates
AFN,Afghanistan
ALL,Albania
AMD,Armenia
ANG,Netherlands Antilles


In [8]:
df_new=pd.merge(df_sum, country, left_index=True, right_index=True,how='left')
df_new=df_new.dropna()
df_new.head()

,Trade_Volume,Exchange_count,Country
Currency,,,
AED,7088,1,United Arab Emirates
ARS,5698,1,Argentina
AUD,22596183431,2,Australia
BRL,18464810423,2,Brazil
CAD,3092002000566944242329053,6,Canada


In [9]:
df['Trade_per_min'] = pd.to_numeric(df['Trade_per_min'], errors='coerce')

In [10]:
df_avg=df.groupby(['Currency'])['Trade_per_min'].agg([np.mean, len])
df_avg.head()

,mean,len
Currency,,
AED,0.010000,1.0
ARS,0.020000,1.0
AUD,0.505000,2.0
BRL,0.870000,2.0
CAD,0.226667,6.0


In [11]:
df_mean=pd.merge(df_avg, country, left_index=True, right_index=True,how='left')
df_mean=df_mean.rename(columns={ df_mean.columns[0]: "Avg_Trade_per_min",df_mean.columns[1]: "Exchange_count"})
df_mean=df_mean.dropna()
df_mean.head()

,Avg_Trade_per_min,Exchange_count,Country
Currency,,,
AED,0.010000,1.0,United Arab Emirates
ARS,0.020000,1.0,Argentina
AUD,0.505000,2.0,Australia
BRL,0.870000,2.0,Brazil
CAD,0.226667,6.0,Canada


In [12]:
df_codes = pd.read_csv('country_codes.csv')
df_codes=df_codes.rename(columns={ df_codes.columns[0]: "Country",df_codes.columns[1]: "Codes"})
df_codes.head()

,Country,Codes
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,American Samoa,ASM
4,Andorra,AND


In [13]:
df_trade=pd.merge(df_mean, df_codes, on='Country', how='left')
df_trade=df_trade[['Country','Codes','Avg_Trade_per_min','Exchange_count']]
df_trade.head()

,Country,Codes,Avg_Trade_per_min,Exchange_count
0,United Arab Emirates,ARE,0.010000,1.0
1,Argentina,ARG,0.020000,1.0
2,Australia,AUS,0.505000,2.0
3,Brazil,BRA,0.870000,2.0
4,Canada,CAN,0.226667,6.0


In [14]:
df_volume=pd.merge(df_new, df_codes, on='Country', how='left')
df_volume=df_volume[['Country','Codes','Trade_Volume','Exchange_count']]
df_volume.head()

,Country,Codes,Trade_Volume,Exchange_count
0,United Arab Emirates,ARE,7088,1
1,Argentina,ARG,5698,1
2,Australia,AUS,22596183431,2
3,Brazil,BRA,18464810423,2
4,Canada,CAN,3092002000566944242329053,6


In [15]:
url = 'https://developers.google.com/public-data/docs/canonical/countries_csv'
latlon = requests.get(url)

soup_l = BeautifulSoup(latlon.text, 'html.parser')
latlogs = soup_l.find_all('table')[0]

df_lat_lon = pd.read_html(str(latlogs))[0]
df_lat_lon=df_lat_lon.rename(columns={ df_lat_lon.columns[1]: "Latitude",df_lat_lon.columns[2]: "Longitude",df_lat_lon.columns[3]: "Country"})
df_lat_lon=df_lat_lon.drop(df_lat_lon.index[0])
df_lat_lon

,0,Latitude,Longitude,Country
1,AD,42.546245,1.601554,Andorra
2,AE,23.424076,53.847818,United Arab Emirates
3,AF,33.93911,67.709953,Afghanistan
4,AG,17.060816,-61.796428,Antigua and Barbuda
5,AI,18.220554,-63.068615,Anguilla
6,AL,41.153332,20.168331,Albania
7,AM,40.069099,45.038189,Armenia
8,AN,12.226079,-69.060087,Netherlands Antilles
9,AO,-11.202692,17.873887,Angola
10,AQ,-75.250973,-0.071389,Antarctica


In [16]:
df_trade_latlong=pd.merge(df_trade, df_lat_lon, on='Country', how='left')
df_trade_latlong.head()

,Country,Codes,Avg_Trade_per_min,Exchange_count,0,Latitude,Longitude
0,United Arab Emirates,ARE,0.010000,1.0,AE,23.424076,53.847818
1,Argentina,ARG,0.020000,1.0,AR,-38.416097,-63.616672
2,Australia,AUS,0.505000,2.0,AU,-25.274398,133.775136
3,Brazil,BRA,0.870000,2.0,BR,-14.235004,-51.92528
4,Canada,CAN,0.226667,6.0,CA,56.130366,-106.346771


In [17]:
df_volume_latlong=pd.merge(df_volume, df_lat_lon, on='Country', how='left')
df_volume_latlong.head()

,Country,Codes,Trade_Volume,Exchange_count,0,Latitude,Longitude
0,United Arab Emirates,ARE,7088,1,AE,23.424076,53.847818
1,Argentina,ARG,5698,1,AR,-38.416097,-63.616672
2,Australia,AUS,22596183431,2,AU,-25.274398,133.775136
3,Brazil,BRA,18464810423,2,BR,-14.235004,-51.92528
4,Canada,CAN,3092002000566944242329053,6,CA,56.130366,-106.346771


In [18]:
df_currency = pd.read_csv('country_currency.csv',encoding='latin-1')
df_currency.head()

,Country,Currency
0,Afghanistan,Afghan afghani
1,Akrotiri and Dhekelia,European euro
2,Aland Islands,European euro
3,Albania,Albanian lek
4,Algeria,Algerian dinar


In [19]:
df_trade_main=pd.merge(df_trade_latlong, df_currency, on='Country', how='left')
df_trade_main.head()

,Country,Codes,Avg_Trade_per_min,Exchange_count,0,Latitude,Longitude,Currency
0,United Arab Emirates,ARE,0.010,1.0,AE,23.424076,53.847818,UAE dirham
1,Argentina,ARG,0.020,1.0,AR,-38.416097,-63.616672,Argentine peso
2,Australia,AUS,0.505,2.0,AU,-25.274398,133.775136,Australian dollar
3,Australia,AUS,0.505,2.0,AU,-25.274398,133.775136,Australian dollar
4,Brazil,BRA,0.870,2.0,BR,-14.235004,-51.92528,Brazilian real


In [23]:
df_volume_main=pd.merge(df_volume_latlong, df_currency, on='Country', how='left')
df_volume_main.head()

,Country,Codes,Trade_Volume,Exchange_count,0,Latitude,Longitude,Currency
0,United Arab Emirates,ARE,7088,1,AE,23.424076,53.847818,UAE dirham
1,Argentina,ARG,5698,1,AR,-38.416097,-63.616672,Argentine peso
2,Australia,AUS,22596183431,2,AU,-25.274398,133.775136,Australian dollar
3,Australia,AUS,22596183431,2,AU,-25.274398,133.775136,Australian dollar
4,Brazil,BRA,18464810423,2,BR,-14.235004,-51.92528,Brazilian real


In [21]:
df_trade_main.to_csv("Trade_over_2Y.csv", encoding='utf-8', index=False)

In [24]:
df_volume_main.to_csv("Volume_over_2Y.csv", encoding='utf-8', index=False)